In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from imutils import paths

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [ ]:
# os.chdir('NewDataset')

In [ ]:
dataset=r'D:\Trical 4th sem\Project\Mask Detector\NewDataset'
# imagePaths=os.listdir(path=)
imagePaths=list(paths.list_images(dataset))


data=[]
labels=[]
# print(imagePaths)

In [ ]:
for i in imagePaths:
    label=i.split(os.path.sep)[-2]
    labels.append(label)
    image=load_img(i,target_size=(224,224))
    image=img_to_array(image)
    image=preprocess_input(image)
    data.append(image)
   





KeyboardInterrupt: 

In [ ]:
data=np.array(data,dtype='float32')
labels=np.array(labels)





In [ ]:
lb=LabelBinarizer()
labels=lb.fit_transform(labels)
labels=to_categorical(labels)
print(labels)




In [ ]:
train_X,test_X,train_Y,test_Y=train_test_split(data,labels,test_size=0.20,stratify=labels,random_state=10)
print(len(train_X))
print(len(test_X))
print(len(train_Y))
print(len(test_Y))
aug=ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,vertical_flip=True,fill_mode='nearest')

baseModel=MobileNetV2(weights='imagenet',include_top=False,input_tensor=Input(shape=(224,224,3)))



In [ ]:
print(baseModel.summary())



In [ ]:
headModel=baseModel.output
headModel=AveragePooling2D(pool_size=(7,7))(headModel)
headModel=Flatten(name='Flatten')(headModel)
headModel=Dense(128,activation='relu')(headModel)
headModel=Dropout(0.5)(headModel)
headModel=Dense(2,activation='softmax')(headModel)

model=Model(inputs=baseModel.input,outputs=headModel)




In [ ]:
for layer in baseModel.layers:
    layer.trainable=False


print(model.summary())

learning_rate=0.001
Epochs=20
BS=12

opt=Adam(lr=learning_rate,decay=learning_rate/Epochs)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])



In [ ]:
H=model.fit(
    aug.flow(train_X,train_Y,batch_size=BS),
    steps_per_epoch=len(train_X)//BS,
    validation_data=(test_X,test_Y),
    validation_steps=len(test_X)//BS,
    epochs=Epochs
)

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array 
from numpy import  expand_dims

In [ ]:
real_testImg = load_img(path='Dataset/2-with-mask.jpg',target_size=(224,224))
real_testImg_Array = img_to_array(real_testImg)
real_testImg_Expanded = expand_dims(real_testImg_Array,axis=0)

In [ ]:
prediction=model.predict(x=real_testImg_Expanded)
print(prediction)